# This notebook implements all the experimentats

In [1]:
# Automatically re-import files when updated
%load_ext autoreload
%autoreload 2  
# Load packages
from models2 import block
from utils import *
from train import run
from evaluate import evaluate
from matplotlib import pyplot as plt
import time, json

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


# Run and evaluate

In [3]:
# Define parameters to be tested
params = [
    [data, adv_attack, epsilon]
    for i, data in enumerate([tf.keras.datasets.mnist.load_data(), tf.keras.datasets.fashion_mnist.load_data(), tf.keras.datasets.cifar10.load_data()])
    for adv_attack in [fgsm, ifgsm]
    for epsilon in [tf.constant(0.01), tf.constant(0.3) if i<2 else tf.constant(0.06)]
]

# Loop over parameters
results = {}
for i, (((X_train, y_train), (X_test, y_test)), adv_attack, epsilon) in enumerate(params):
    # Report current iteration
    dataname = "MNIST" if i < 4 else "FMNIST" if i < 8 else "CIFAR"
    current = f"{adv_attack.__name__} - {float(epsilon)} - {dataname}"
    print(current)

    # Convert data to tf dataset 
    data_train = color_dataset_from_arrays(X_train, y_train) if dataname=="CIFAR" else gray_dataset_from_arrays(X_train, y_train)
    data_test = color_dataset_from_arrays(X_test, y_test) if dataname=="CIFAR" else gray_dataset_from_arrays(X_test, y_test)

    # Train the models and save them
    run(data_train, adv_attack, epsilon)

    # Load models
    with tf.keras.utils.custom_object_scope({
        "block": block
        }):
        decoder = tf.keras.models.load_model("decoder.keras")
        encoder = tf.keras.models.load_model("encoder.keras")
        classifier = tf.keras.models.load_model("classifier.keras")

    # Evaluate
    accuracies = evaluate(data_test, len(X_train)//BATCH_SIZE, encoder, decoder, classifier, adv_attack, epsilon)

    # Save results
    results[current] = accuracies
    json.dump(results, open("results.json", mode="w"))

ValueError: too many values to unpack (expected 2)